In [27]:
import os

from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
import googleapiclient.discovery
import googleapiclient.errors
import google_auth_oauthlib.flow

scopes = ["https://www.googleapis.com/auth/youtube.force-ssl"]

flow = InstalledAppFlow.from_client_secrets_file(
 "Youtube_Oauth.json", scopes = scopes)

flow.run_local_server(port = 8080)

credentials = flow.credentials

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=725695838850-3pp8b4qjgqebrq8iqdag2pmkhcmp7h3i.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=rbNuTBczYhnb2FHCza40ItxE7EEJXR&access_type=offline


In [29]:
api_service_name = "youtube"
api_version = "v3"
client_secrets_file = "Youtube_Oauth.json"

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, credentials=credentials)

In [30]:
#Healthcare machine learning lesson id

hc_ml_id = ['vof7x8r_ZUA','DS97JV_o0Fs','0UFwGJe6ubg','_shuV1tJbTU','wqI_z1yumzY',
           'lLhfDSOwWtU','IiD3YZkkCmE','lkO2ocJBsmI','ZQu2B3GyI_k','MoEaRpLNo9A',
           'VuKOW8d4KHw','PKCMH5KOcxQ','2ZXYM1h9pgY','gRkUhg9Wb-I','g5v-NvNoJQQ',
           'YZ5pOgY5hEE','zdotUAxiPGM','yYWyLZrdRRI','aJqgO8e37_g','kZrb6ZIwJqg',
           'Td01vFP3uJo','k95abdkdCPk','zYgkr0KfWM0','MdUnh4PaGKw','wDLzLN1tArA']

In [31]:
#cancer drug id



drug_id = ['rtR3yl5U4yk','zm3S71Gk1qE','QbcxlOalFr0','wytDunVxNx0','fecZsFwETII']
#QbcxlOalFr0: Anti-inflammatory drugs 
#wytDunVxNx0: drug abuse
#fecZsFwETII: Ibuprofen vs. Aleve vs. Turmeric vs. Tylenol (Updated with Aspirin) Pharmacist Chris Explains



In [32]:
def caption_merge(text,video_id,ans):
    #Get youtube info
    request = youtube.videos().list(
    part="snippet",
    id=video_id)
    response = request.execute()
    title = response['items'][0]['snippet']['title']
    channelTitle = response['items'][0]['snippet']['channelTitle']
    publishedAt = response['items'][0]['snippet']['publishedAt']
    
    
    
    output = []
    curr_caption = ' '
    current_id = 0

    for i in range(len(text)):
        #final_stop
        if i == len(text)-1:
            output.append({'text':curr_caption,
                          'id':video_id+'_'+str(int(current_id))})
            pass
        
        #to next session
        if current_id != text[i]['start']//60:
            output.append({'text':curr_caption,
                          'id':video_id+'_'+str(int(current_id)),
                          'title':title,
                          channelTitle:channelTitle,
                          publishedAt:publishedAt})
            
            current_id = text[i]['start']//60
            curr_caption = ' '
        
        else:
            curr_caption += (' ' + text[i]['text'])
            continue
    
    ans += output

In [33]:
from youtube_transcript_api import YouTubeTranscriptApi

ans_hc_ml = []
ans_drug = []

for i in range(len(hc_ml_id)):
    caption = YouTubeTranscriptApi.get_transcript(hc_ml_id[i])
    caption_merge(caption,hc_ml_id[i],ans_hc_ml)

    
for i in range(len(drug_id)):
    caption = YouTubeTranscriptApi.get_transcript(drug_id[i])
    caption_merge(caption,hc_ml_id[i],ans_drug)


In [34]:
import json

# Sample dictionary data
data = {
    "HC-ML":ans_hc_ml,
    "drug":ans_drug
}

# Save the dictionary as a JSON file
with open("Healthcare-ML-caption.json", "w") as file:
    json.dump(data, file)